In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# --- Parameters ---#
IMAGE_SIZE = (224, 224)
IMG_SHAPE = IMAGE_SIZE + (3,)
NUM_CLASSES = 2

In [ ]:
# --- Focal Loss Function (must match training) ---#
def focal_loss(gamma=2.0, alpha=0.75):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow(1 - y_pred, gamma)
        return tf.reduce_sum(weight * cross_entropy, axis=1)
    return loss

In [ ]:
# --- Rebuild Model Architecture ---#
def build_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    base_model = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=IMG_SHAPE,
        pooling='avg'
    )(inputs)
    x = layers.Dropout(0.5)(base_model)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
# --- Load Model and Weights ---#
model = build_model()
model.load_weights('/kaggle/input/test12/gender_classifier_full.h5')

In [ ]:
# --- Load the saved threshold ---#
with open('/kaggle/input/test12/female_threshold.txt', 'r') as f:
    female_thresh = float(f.read().strip())

In [ ]:
# --- Function to Preprocess and Predict on Test Images ---#
def load_and_preprocess_image(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=IMAGE_SIZE)
    img = tf.keras.utils.img_to_array(img)
    img = img / 255.0
    return img

In [ ]:
def print_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    # Calculate max length for formatting
    max_len = max(len(name) for name in class_names) + 5
    
    # Create header
    header = " " * max_len + "| " + " | ".join([f"Predicted {name}" for name in class_names])
    separator = "-" * len(header)
    
    # Create rows
    rows = []
    for i, true_name in enumerate(class_names):
        row = f"True {true_name}".ljust(max_len) + "| "
        row += " | ".join([f"{cm[i,j]:<{len('Predicted ' + class_names[j])}}" for j in range(len(class_names))])
        rows.append(row)
    
    # Print matrix
    print("\nConfusion Matrix:")
    print(header)
    print(separator)
    for row in rows:
        print(row)

In [ ]:
def predict_and_evaluate(female_folder, male_folder, female_thresh):
    y_true = []
    y_pred = []
    results = []

    # Female images
    image_files = [f for f in os.listdir(female_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for fname in image_files:
        img_path = os.path.join(female_folder, fname)
        img = load_and_preprocess_image(img_path)
        img_batch = np.expand_dims(img, axis=0)
        pred = model.predict(img_batch, verbose=0)
        female_prob = pred[0][0]
        pred_class = 0 if female_prob >= female_thresh else 1  # 0: Female, 1: Male
        y_true.append(0)  # 0: Female
        y_pred.append(pred_class)
        results.append({
            'filename': fname,
            'true_class': 'Female',
            'predicted_class': 'Female' if pred_class == 0 else 'Male',
            'prob_female': float(pred[0][0]),
            'prob_male': float(pred[0][1])
        })

    # Male images
    image_files = [f for f in os.listdir(male_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for fname in image_files:
        img_path = os.path.join(male_folder, fname)
        img = load_and_preprocess_image(img_path)
        img_batch = np.expand_dims(img, axis=0)
        pred = model.predict(img_batch, verbose=0)
        female_prob = pred[0][0]
        pred_class = 0 if female_prob >= female_thresh else 1  # 0: Female, 1: Male
        y_true.append(1)  # 1: Male
        y_pred.append(pred_class)
        results.append({
            'filename': fname,
            'true_class': 'Male',
            'predicted_class': 'Female' if pred_class == 0 else 'Male',
            'prob_female': float(pred[0][0]),
            'prob_male': float(pred[0][1])
        })
    return y_true, y_pred, results

In [ ]:
female_folder = '/kaggle/input/male-and-female-faces-dataset/Male and Female face dataset/Female Faces'  # PUT TEST FILES For FEMALE HERE
male_folder = '/kaggle/input/male-and-female-faces-dataset/Male and Female face dataset/Male Faces'      # or PUT TEST FILES For MALE HERE

y_true, y_pred, results = predict_and_evaluate(female_folder, male_folder, female_thresh)

# Print classification report
class_names = ['Female', 'Male']
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

# Print confusion matrix
print_confusion_matrix(y_true, y_pred, class_names)